This is the torch to tensorrt converter notebook. I will add more stuff once I know what to add. I am following this link: https://learnopencv.com/how-to-convert-a-model-from-pytorch-to-tensorrt-and-speed-up-inference/

In [1]:
import torch
import os
from torchvision.models import resnet50, ResNet50_Weights
import torch.nn as nn

In [2]:
trained_model_path = "../runs/traversky/2023_03_05_21_30_33/model.pth"
ONNX_FILE_PATH = 'bin_classifier.onnx'
tensorrt_engine_path = "engine.trt"

In [3]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(2048, 2)

In [4]:
# model loaded
checkpoint = torch.load(trained_model_path)
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [6]:
# model

Now the tensorrt conversion. 

First step is to make .onnx file.  

In [7]:
dummy_input = torch.empty((1, 3, 256, 256))
torch.onnx.export(model, dummy_input, ONNX_FILE_PATH, input_names=['input'],
                  output_names=['output'], export_params=True)

Now lets see the tensor rt

In [8]:
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
import tensorrt as trt

In [9]:
# # Get logger

# def build_engine(onnx_file_path):
#     # initialize TensorRT engine and parse ONNX model
#     TRT_LOGGER = trt.Logger()
#     builder = trt.Builder(TRT_LOGGER)
#     network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))
#     profile = builder.create_optimization_profile()
#     config = builder.create_builder_config()
    
#     # Setting engine_precesion to FP16
#     config.set_flag(trt.BuilderFlag.FP16)
    
#     # parse ONNX
#     parser = trt.OnnxParser(network, TRT_LOGGER)
    
#     with open(onnx_file_path, 'rb') as model:
#         print('Beginning ONNX file parsing')
#         parser.parse(model.read())
#     print('Completed parsing of ONNX file')
#     inputTensor = network.get_input(0)
#     print('inputTensor.name:', inputTensor.name)
#     network.unmark_output(network.get_output(0))

#     print('Building an engine...')
#     engineString = builder.build_serialized_network(network, config)
#     type(engineString)
# #     with open(tensorrt_enginepath, "wb") as f:
# #         f.write(engineString)
     
#     #     context = engine.create_execution_context()
# #     print("Completed creating Engine")
 
# #     return engine, context

In [12]:
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
def build_engine(onnx_path, shape = [1,224,224,3]):

   """
   This is the function to create the TensorRT engine
   Args:
      onnx_path : Path to onnx_file. 
      shape : Shape of the input of the ONNX file. 
  """
   with trt.Builder(TRT_LOGGER) as builder, builder.create_network(1) as network, builder.create_builder_config() as config, trt.OnnxParser(network, TRT_LOGGER) as parser:
       config.max_workspace_size = (256 << 20)
       with open(onnx_path, 'rb') as model:
           parser.parse(model.read())
       network.get_input(0).shape = shape
       engine = builder.build_engine(network, config)
       return engine

def save_engine(engine, file_name):
   buf = engine.serialize()
   with open(file_name, 'wb') as f:
       f.write(buf)

def load_engine(trt_runtime, plan_path):
   with open(plan_path, 'rb') as f:
       engine_data = f.read()
   engine = trt_runtime.deserialize_cuda_engine(engine_data)
   return engine

In [16]:
import argparse
import tensorrt as trt 

engine_name = "resnet50.plan"
batch_size = 1 

shape = [batch_size , 3, 256 ,256]
engine = build_engine(ONNX_FILE_PATH, shape= shape)
save_engine(engine, engine_name) 

[06/12/2023-12:26:12] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars


/tmp/ipykernel_18519/2683552468.py:11: DeprecationWarning: Use set_memory_pool_limit instead.
  config.max_workspace_size = (256 << 20)
/tmp/ipykernel_18519/2683552468.py:15: DeprecationWarning: Use build_serialized_network instead.
  engine = builder.build_engine(network, config)


[06/12/2023-12:26:13] [TRT] [W] TensorRT was linked against cuDNN 8.6.0 but loaded cuDNN 8.3.2
[06/12/2023-12:26:24] [TRT] [W] TensorRT was linked against cuDNN 8.6.0 but loaded cuDNN 8.3.2
